In [1]:
import pandas as pd
import os
from ydata_profiling import ProfileReport
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib notebook

import plotly.express as px
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC

/opt/conda/lib/python3.10/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
/opt/conda/lib/python3.10/site-packages/visions/backends/shared/nan_handling.py:51: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def hasna(x: np.ndarray) -> bool:


In [2]:
os.listdir('/kaggle/input/')
# Wczytanie danych
all_df = pd.read_csv('../input/mushroom-classification/mushrooms.csv')
all_df.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


Ekploracyjna analiza danych

In [3]:
all_df.describe().T

,count,unique,top,freq
class,8124,2,e,4208
cap-shape,8124,6,x,3656
cap-surface,8124,4,y,3244
cap-color,8124,10,n,2284
bruises,8124,2,f,4748
odor,8124,9,n,3528
gill-attachment,8124,2,f,7914
gill-spacing,8124,2,c,6812
gill-size,8124,2,b,5612
gill-color,8124,12,b,1728


In [4]:
all_df.shape

(8124, 23)

In [5]:
all_df.isnull().sum()

class                       0
cap-shape                   0
cap-surface                 0
cap-color                   0
bruises                     0
odor                        0
gill-attachment             0
gill-spacing                0
gill-size                   0
gill-color                  0
stalk-shape                 0
stalk-root                  0
stalk-surface-above-ring    0
stalk-surface-below-ring    0
stalk-color-above-ring      0
stalk-color-below-ring      0
veil-type                   0
veil-color                  0
ring-number                 0
ring-type                   0
spore-print-color           0
population                  0
habitat                     0
dtype: int64

In [6]:
all_df['habitat'].groupby(all_df['class']).count()

class
e    4208
p    3916
Name: habitat, dtype: int64

In [7]:
all_df.shape

(8124, 23)

In [8]:
all_df.isnull().sum()

class                       0
cap-shape                   0
cap-surface                 0
cap-color                   0
bruises                     0
odor                        0
gill-attachment             0
gill-spacing                0
gill-size                   0
gill-color                  0
stalk-shape                 0
stalk-root                  0
stalk-surface-above-ring    0
stalk-surface-below-ring    0
stalk-color-above-ring      0
stalk-color-below-ring      0
veil-type                   0
veil-color                  0
ring-number                 0
ring-type                   0
spore-print-color           0
population                  0
habitat                     0
dtype: int64

In [9]:
all_df['habitat'].groupby(all_df['class']).count()

class
e    4208
p    3916
Name: habitat, dtype: int64

In [10]:
df_habitat = all_df['habitat'].groupby(all_df['class']).value_counts(normalize= True).rename('frequency').to_frame().reset_index()
df_habitat

,class,habitat,frequency
0,e,d,0.446768
1,e,g,0.334601
2,e,m,0.060837
3,e,l,0.057034
4,e,w,0.045627
5,e,p,0.032319
6,e,u,0.022814
7,p,d,0.323800
8,p,p,0.257406
9,p,g,0.188968


In [11]:
df_population= all_df["population"].groupby(all_df["class"]).value_counts(normalize= True).rename("frequency").to_frame().reset_index()
df_population

,class,population,frequency
0,e,v,0.283270
1,e,y,0.252852
2,e,s,0.209125
3,e,n,0.095057
4,e,a,0.091255
5,e,c,0.068441
6,p,v,0.727273
7,p,y,0.165475
8,p,s,0.093973
9,p,c,0.013279


In [12]:
df_population= all_df["gill-spacing"].groupby(all_df["class"]).value_counts(normalize= True).rename("frequency").to_frame().reset_index()
df_population

,class,gill-spacing,frequency
0,e,c,0.714829
1,e,w,0.285171
2,p,c,0.971399
3,p,w,0.028601


In [13]:
all_df

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,e,k,s,n,f,n,a,c,b,y,...,s,o,o,p,o,o,p,b,c,l
8120,e,x,s,n,f,n,a,c,b,y,...,s,o,o,p,n,o,p,b,v,l
8121,e,f,s,n,f,n,a,c,b,n,...,s,o,o,p,o,o,p,b,c,l
8122,p,k,y,n,f,y,f,c,n,b,...,k,w,w,p,w,o,e,w,v,l


In [14]:
le = LabelEncoder()
col =['class', 'bruises',  'gill-spacing', 'gill-size', 'stalk-shape']
for i in col:
    all_df[i] = all_df[i].astype('category')
    all_df[i] = all_df[i].cat.codes

In [15]:
all_df.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,x,s,n,1,p,f,0,1,k,...,s,w,w,p,w,o,p,k,s,u
1,0,x,s,y,1,a,f,0,0,k,...,s,w,w,p,w,o,p,n,n,g
2,0,b,s,w,1,l,f,0,0,n,...,s,w,w,p,w,o,p,n,n,m
3,1,x,y,w,1,p,f,0,1,n,...,s,w,w,p,w,o,p,k,s,u
4,0,x,s,g,0,n,f,1,0,k,...,s,w,w,p,w,o,e,n,a,g


In [16]:
all_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   class                     8124 non-null   int8  
 1   cap-shape                 8124 non-null   object
 2   cap-surface               8124 non-null   object
 3   cap-color                 8124 non-null   object
 4   bruises                   8124 non-null   int8  
 5   odor                      8124 non-null   object
 6   gill-attachment           8124 non-null   object
 7   gill-spacing              8124 non-null   int8  
 8   gill-size                 8124 non-null   int8  
 9   gill-color                8124 non-null   object
 10  stalk-shape               8124 non-null   int8  
 11  stalk-root                8124 non-null   object
 12  stalk-surface-above-ring  8124 non-null   object
 13  stalk-surface-below-ring  8124 non-null   object
 14  stalk-color-above-ring  

In [17]:
print("Kolumny z danymi kategorialnymi : ")
category_col = list(all_df.select_dtypes(include = ['object']).columns)
print(category_col)

Kolumny z danymi kategorialnymi : 
['cap-shape', 'cap-surface', 'cap-color', 'odor', 'gill-attachment', 'gill-color', 'stalk-root', 'stalk-surface-above-ring', 'stalk-surface-below-ring', 'stalk-color-above-ring', 'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number', 'ring-type', 'spore-print-color', 'population', 'habitat']


In [18]:
df = pd.get_dummies(all_df, drop_first =True)

df.head()

,class,bruises,gill-spacing,gill-size,stalk-shape,cap-shape_c,cap-shape_f,cap-shape_k,cap-shape_s,cap-shape_x,...,population_n,population_s,population_v,population_y,habitat_g,habitat_l,habitat_m,habitat_p,habitat_u,habitat_w
0,1,1,0,1,0,False,False,False,False,True,...,False,True,False,False,False,False,False,False,True,False
1,0,1,0,0,0,False,False,False,False,True,...,True,False,False,False,True,False,False,False,False,False
2,0,1,0,0,0,False,False,False,False,False,...,True,False,False,False,False,False,True,False,False,False
3,1,1,0,1,0,False,False,False,False,True,...,False,True,False,False,False,False,False,False,True,False
4,0,0,1,0,1,False,False,False,False,True,...,False,False,False,False,True,False,False,False,False,False


In [19]:
X = df.drop("class",axis=1)
y = df["class"]

In [20]:
X

,bruises,gill-spacing,gill-size,stalk-shape,cap-shape_c,cap-shape_f,cap-shape_k,cap-shape_s,cap-shape_x,cap-surface_g,...,population_n,population_s,population_v,population_y,habitat_g,habitat_l,habitat_m,habitat_p,habitat_u,habitat_w
0,1,0,1,0,False,False,False,False,True,False,...,False,True,False,False,False,False,False,False,True,False
1,1,0,0,0,False,False,False,False,True,False,...,True,False,False,False,True,False,False,False,False,False
2,1,0,0,0,False,False,False,False,False,False,...,True,False,False,False,False,False,True,False,False,False
3,1,0,1,0,False,False,False,False,True,False,...,False,True,False,False,False,False,False,False,True,False
4,0,1,0,1,False,False,False,False,True,False,...,False,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,0,0,0,0,False,False,True,False,False,False,...,False,False,False,False,False,True,False,False,False,False
8120,0,0,0,0,False,False,False,False,True,False,...,False,False,True,False,False,True,False,False,False,False
8121,0,0,0,0,False,True,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
8122,0,0,1,1,False,False,True,False,False,False,...,False,False,True,False,False,True,False,False,False,False


In [21]:
y

0       1
1       0
2       0
3       1
4       0
       ..
8119    0
8120    0
8121    0
8122    1
8123    0
Name: class, Length: 8124, dtype: int8

Podzielenie danych na zbiory

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

In [23]:
X.shape ,X_train.shape, X_test.shape

((8124, 95), (7311, 95), (813, 95))

In [24]:
X_train.head()

,bruises,gill-spacing,gill-size,stalk-shape,cap-shape_c,cap-shape_f,cap-shape_k,cap-shape_s,cap-shape_x,cap-surface_g,...,population_n,population_s,population_v,population_y,habitat_g,habitat_l,habitat_m,habitat_p,habitat_u,habitat_w
4750,0,0,0,0,False,True,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
3026,0,1,1,0,False,False,False,False,True,False,...,False,True,False,False,False,False,False,False,False,False
3757,1,0,0,1,False,True,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
6167,0,0,1,1,False,True,False,False,False,False,...,False,False,True,False,False,False,False,True,False,False
6281,0,0,1,1,False,False,False,False,True,False,...,False,False,True,False,False,True,False,False,False,False


Trenowanie modeli i walidacja:

In [25]:
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)

DecisionTreeClassifier()

In [26]:
train_pred_dtc= dtc.predict(X_train)
test_pred_dtc = dtc.predict(X_test)

In [27]:
train_acc_dtc = accuracy_score(y_train, train_pred_dtc)
test_acc_dtc = accuracy_score(y_test, test_pred_dtc)

print(f"Train Set Accuracy: {train_acc_dtc}")
print(f"Test Set Accuracy: {test_acc_dtc}")


print("\n\nTrain Set Confusion Matrix\n")
print(confusion_matrix(y_train, train_pred_dtc))

print("\n\nTest Set Confusion Matrix\n")
print(confusion_matrix(y_test, test_pred_dtc))

Train Set Accuracy: 1.0
Test Set Accuracy: 1.0


Train Set Confusion Matrix

[[3775    0]
 [   0 3536]]


Test Set Confusion Matrix

[[433   0]
 [  0 380]]


In [28]:
print(f"Train Set Report:\n\n {classification_report(y_train, train_pred_dtc)}")
print(f"Test Set Report:\n\n {classification_report(y_test, test_pred_dtc)}")

Train Set Report:

               precision    recall  f1-score   support

           0       1.00      1.00      1.00      3775
           1       1.00      1.00      1.00      3536

    accuracy                           1.00      7311
   macro avg       1.00      1.00      1.00      7311
weighted avg       1.00      1.00      1.00      7311

Test Set Report:

               precision    recall  f1-score   support

           0       1.00      1.00      1.00       433
           1       1.00      1.00      1.00       380

    accuracy                           1.00       813
   macro avg       1.00      1.00      1.00       813
weighted avg       1.00      1.00      1.00       813



In [29]:
m2 = "k-Nearest Neighbors"
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [30]:
train_pred_knn= knn.predict(X_train)
test_pred_knn = knn.predict(X_test)

In [31]:
train_acc_knn = accuracy_score(y_train, train_pred_knn)
test_acc_knn = accuracy_score(y_test, test_pred_knn)

print(f"Train Set Accuracy: {train_acc_knn}")
print(f"Test Set Accuracy: {test_acc_knn}")



print("\n\nTrain Set Confusion Matrix\n")
print(confusion_matrix(y_train, train_pred_knn))

print("\n\nTest Set Confusion Matrix\n")
print(confusion_matrix(y_test, test_pred_knn))

Train Set Accuracy: 1.0
Test Set Accuracy: 1.0


Train Set Confusion Matrix

[[3775    0]
 [   0 3536]]


Test Set Confusion Matrix

[[433   0]
 [  0 380]]


In [32]:
print(f"Train Set Report:\n\n {classification_report(y_train, train_pred_knn)}")
print(f"Test Set Report:\n\n {classification_report(y_test, test_pred_knn)}")

Train Set Report:

               precision    recall  f1-score   support

           0       1.00      1.00      1.00      3775
           1       1.00      1.00      1.00      3536

    accuracy                           1.00      7311
   macro avg       1.00      1.00      1.00      7311
weighted avg       1.00      1.00      1.00      7311

Test Set Report:

               precision    recall  f1-score   support

           0       1.00      1.00      1.00       433
           1       1.00      1.00      1.00       380

    accuracy                           1.00       813
   macro avg       1.00      1.00      1.00       813
weighted avg       1.00      1.00      1.00       813



In [33]:
for k in range(1, 11):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    test_pred_knn = knn.predict(X_test)
    acc = accuracy_score(y_test, test_pred_knn)
    print(f"n_neighbors={k}, Test Set Accuracy: {acc}")

n_neighbors=1, Test Set Accuracy: 1.0
n_neighbors=2, Test Set Accuracy: 1.0
n_neighbors=3, Test Set Accuracy: 1.0
n_neighbors=4, Test Set Accuracy: 1.0
n_neighbors=5, Test Set Accuracy: 1.0
n_neighbors=6, Test Set Accuracy: 1.0
n_neighbors=7, Test Set Accuracy: 1.0
n_neighbors=8, Test Set Accuracy: 1.0
n_neighbors=9, Test Set Accuracy: 1.0
n_neighbors=10, Test Set Accuracy: 1.0


In [34]:
metrics = ['euclidean', 'manhattan']

for metric in metrics:
    knn = KNeighborsClassifier(metric=metric)
    knn.fit(X_train, y_train)
    test_pred_knn = knn.predict(X_test)
    acc = accuracy_score(y_test, test_pred_knn)
    print(f"Metric={metric}, Test Set Accuracy: {acc}")

Metric=euclidean, Test Set Accuracy: 1.0
Metric=manhattan, Test Set Accuracy: 1.0


In [35]:
for depth in range(1, 11):
    dtc = DecisionTreeClassifier(max_depth=depth)
    dtc.fit(X_train, y_train)
    test_pred_dtc = dtc.predict(X_test)
    acc = accuracy_score(y_test, test_pred_dtc)
    print(f"max_depth={depth}, Test Set Accuracy: {acc}")

max_depth=1, Test Set Accuracy: 0.8880688806888068
max_depth=2, Test Set Accuracy: 0.955719557195572
max_depth=3, Test Set Accuracy: 0.985239852398524
max_depth=4, Test Set Accuracy: 0.992619926199262
max_depth=5, Test Set Accuracy: 0.998769987699877
max_depth=6, Test Set Accuracy: 0.998769987699877
max_depth=7, Test Set Accuracy: 1.0
max_depth=8, Test Set Accuracy: 1.0
max_depth=9, Test Set Accuracy: 1.0
max_depth=10, Test Set Accuracy: 1.0


In [36]:
criteria = ['gini', 'entropy']

for criterion in criteria:
    dtc = DecisionTreeClassifier(criterion=criterion)
    dtc.fit(X_train, y_train)
    test_pred_dtc = dtc.predict(X_test)
    acc = accuracy_score(y_test, test_pred_dtc)
    print(f"Criterion={criterion}, Test Set Accuracy: {acc}")

Criterion=gini, Test Set Accuracy: 1.0
Criterion=entropy, Test Set Accuracy: 1.0
